In [1]:
import mlflow.sklearn
from mlflow import log_param
from mlflow.tracking import MlflowClient

import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from sklearn.metrics import classification_report, f1_score, roc_auc_score

from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import EditedNearestNeighbours

from lightgbm import LGBMClassifier

In [2]:
dataset = pd.read_csv('data/dataset_5818_artscore.csv')

/home/guillaume/code/GGIML/music_project/musenv/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
dataset

,name,album,artist,release_date,popularity,duration_ms,key,mode,time_signature,acousticness,...,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,year,target,artist_score
0,Corner Man - Explicit Album Version,Last 2 Walk,Three 6 Mafia,2008-06-21,20,187480,2,1,4,0.080300,...,0.783,0.000000,0.0786,-5.480,0.0720,0.274,157.057,2008,0.0,4.0
1,Old Judge Jones,Say No More,Les Dudek,1977-01-01,29,278800,8,1,4,0.094400,...,0.647,0.004810,0.2470,-9.260,0.0343,0.652,135.866,1977,0.0,0.0
2,Endless Fantasy,Endless Fantasy,Anamanaguchi,2013-05-14,43,357785,8,1,3,0.000315,...,0.731,0.025200,0.2210,-4.027,0.1030,0.293,126.692,2013,0.0,0.0
3,Hope Has a Place,The Memory of Trees,Enya,1995-11-20,35,288640,11,1,4,0.954000,...,0.127,0.092700,0.1090,-18.247,0.0383,0.071,70.883,1995,0.0,2.0
4,Put the Needle on It - Radio Version,Neon Nights (Deluxe Version),Dannii Minogue,2003-01-01,37,204000,6,0,4,0.014100,...,0.618,0.035800,0.0970,-8.194,0.0651,0.776,120.021,2003,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208821,What You Get Is What You See - Live,Tina Live in Europe,Tina Turner,1988-03-16,32,334040,5,1,4,0.006120,...,0.906,0.001860,0.9880,-10.985,0.0610,0.676,168.447,1988,1.0,7.0
208822,What You Waiting For?,Love Angel Music Baby (Deluxe Version),Gwen Stefani,2004-11-23,54,221227,5,1,4,0.050900,...,0.948,0.000008,0.3840,-2.557,0.0628,0.731,136.027,2004,1.0,0.0
208823,Say What You Want,The Greatest Hits,Texas,2000-10-23,44,230547,4,1,4,0.176000,...,0.824,0.000000,0.1150,-6.015,0.0753,0.777,95.893,2000,1.0,1.0
208824,What's a Guy Gotta Do,What's a Guy Gotta Do,John McNicholl,2005-09-02,17,147987,1,1,4,0.592000,...,0.778,0.000000,0.0897,-7.352,0.0318,0.968,169.910,2005,1.0,0.0


data_2018 = dataset[dataset.year==2018]

In [4]:
data_new = dataset.drop(['name', 'album', 'artist', 'popularity', 'release_date'], axis=1)

/home/guillaume/code/GGIML/music_project/musenv/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
data_new.to_csv('dataset_vf')

In [6]:
to_encode=['year', 'key', 'mode']
to_bin=['duration_ms']
num_cols = ['time_signature', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence', 'tempo', 'artist_score']

In [7]:
transformer = ColumnTransformer([('scaler', StandardScaler(), num_cols),
                                 ("discretizer", KBinsDiscretizer(), to_bin),
                                 ("encoder", OneHotEncoder(drop='first'), to_encode)],
                                n_jobs=-1, verbose=1)

In [8]:
X = data_new.drop('target', axis=1)
y = data_new.target

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42, stratify=y)

In [10]:
X_train = transformer.fit_transform(X_train)

/home/guillaume/code/GGIML/music_project/musenv/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
X_test = transformer.transform(X_test)

/home/guillaume/code/GGIML/music_project/musenv/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
other_cols = [col for col in num_cols if col not in to_bin+to_encode]
bin_edges = transformer.transformers_[1][1].bin_edges_[0]
onehot_columns = transformer.transformers_[2][1].get_feature_names(to_encode)

/home/guillaume/code/GGIML/music_project/musenv/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
features_transformed = other_cols+\
[f'Duration:[{bin_edges[i]}:{bin_edges[i+1]}]' for i in range(len(bin_edges)-1)]+\
list(onehot_columns)

In [14]:
def create_experiment(experiment_name) -> int:
    """
    Creates an mlflow experiment
    :param experiment_name: str. The name of the experiment to be set in MLFlow
    :return: the id of the experiment created if it doesn't exist, or the id of the existing experiment if it is already
    :return: the id of the experiment created if it doesn't exist, or the id of the existing experiment if it is already
    created
    """
    client = MlflowClient()
    experiments = client.list_experiments()
    experiment_names = set(map(lambda e: e.name, experiments))
    if experiment_name in experiment_names:
        print(f'Experiment {experiment_name} already created.')
        return list(filter(lambda e: e.name == experiment_name, experiments))[0].experiment_id
    else:
        return mlflow.create_experiment(name=experiment_name)

def eval_metrics(actual, pred):
    f1 = f1_score(actual, pred, pos_label=1)
    roc_auc = roc_auc_score(actual, pred)
    return f1, roc_auc

In [15]:
for sampling_strat in np.arange(0.2, 1, 0.1):

    experiment_id = create_experiment(experiment_name='SMOTE LGBM')

    with mlflow.start_run(experiment_id=experiment_id, nested=False):
        mlflow.set_tracking_uri("http://127.0.0.1:5000")

        mlflow.lightgbm.autolog()

        over_sampler = SMOTE(sampling_strategy=sampling_strat)
        under_sampler = EditedNearestNeighbours()
        smote = SMOTEENN(smote=over_sampler, enn=under_sampler,random_state=42)

        X_train_sampled, y_train_sampled = smote.fit_sample(X_train, y_train)

        #Train a machine learning model of your choice with scikit learn
        lgbm = LGBMClassifier(max_depth=3, num_leaves=5, n_estimators=300, random_state=42)
        lgbm.fit(X_train_sampled, y_train_sampled)

        #Make prediction
        y_pred_lgbm = lgbm.predict(X_test)
        y_pred_train_lgbm = lgbm.predict(X_train)

        #Look at some metrics like the rmse
        f1_test, roc_auc_test = eval_metrics(y_test, y_pred_lgbm)
        f1_train, roc_auc_train = eval_metrics(y_train, y_pred_train_lgbm)

        #Now your can log the params and metrics of your model
        mlflow.log_param('max_depth', lgbm.max_depth)
        mlflow.log_param('num_leaves', lgbm.num_leaves)
        mlflow.log_param('n_estimators', lgbm.n_estimators)
        mlflow.log_metric('F1_test', f1_test)
        mlflow.log_metric('F1_train', f1_train)
        mlflow.log_metric('ROC_AUC_test', roc_auc_test)
        mlflow.log_metric('ROC_AUC_train', roc_auc_train)
        mlflow.log_metric('Resampling_strat', sampling_strat)

        #Log your scikit learn model
        #mlflow.lightgbm.log_model(lgbm, 'lgbm')

Experiment SMOTE LGBM already created.
Experiment SMOTE LGBM already created.


KeyboardInterrupt: 

In [ ]:
lgbm = LGBMClassifier(max_depth=3, num_leaves=5, n_estimators=300, random_state=42)

In [ ]:
lgbm.fit(X_train_sampled, y_train_sampled)

In [ ]:
y_pred_lgbm = lgbm.predict(X_test)
y_pred_train_lgbm = lgbm.predict(X_train)

In [ ]:
print(classification_report(y_test, y_pred_lgbm))

In [ ]:
print(classification_report(y_train, y_pred_train_lgbm))

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid_lgbm = {'num_leaves':[15, 20, 25],
                   'max_depth':[3, 5, 7],
                   'n_estimators':[100, 200, 300]}

In [ ]:
grid_lgbm = GridSearchCV(lgbm, param_grid=param_grid_lgbm, n_jobs=-1, scoring='f1_macro')

In [ ]:
grid_lgbm.fit(X_train_sampled, y_train_sampled)

In [ ]:
grid_lgbm.best_params_

In [ ]:
best_lgbm = grid_lgbm.best_estimator_

In [ ]:
y_pred_lgbm = best_lgbm.predict(X_test)
y_pred_train_lgbm = best_lgbm.predict(X_train)

In [ ]:
print(classification_report(y_test, y_pred_lgbm))

In [ ]:
print(classification_report(y_train, y_pred_train_lgbm))

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logr = LogisticRegression(random_state=42)

In [ ]:
logr.fit(X_train_sampled, y_train_sampled)

In [ ]:
y_pred = logr.predict(X_test)

In [ ]:
y_pred_train = logr.predict(X_train)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
print(classification_report(y_train, y_pred_train))

In [ ]:
param_grid_logr = {'solver':['newton-cg', 'lbfgs', 'saga'],
                  'penalty':['l1', 'l2', 'elasticnet']}

In [ ]:
grid_logr = GridSearchCV(logr, param_grid=param_grid_logr, n_jobs=-1, scoring='f1_macro')

In [ ]:
grid_logr.fit(X_train_sampled, y_train_sampled)

In [ ]:
grid_logr.best_params_

In [ ]:
logr = grid_logr.best_estimator_

In [ ]:
param_grid_logr = {'C':[0.1, 1, 10, 100, 1000]}

In [ ]:
grid_logr = GridSearchCV(logr, param_grid=param_grid_logr, n_jobs=-1, scoring='f1_macro')

In [ ]:
grid_logr.fit(X_train_sampled, y_train_sampled)

In [ ]:
grid_logr.best_params_

In [ ]:
param_grid_logr = {'C':np.random.normal(1, 1, 1000)}

In [ ]:
grid_logr = GridSearchCV(logr, param_grid=param_grid_logr, n_jobs=-1, scoring='f1_macro')

In [ ]:
grid_logr.fit(X_train_sampled, y_train_sampled)

In [ ]:
grid_logr.best_params_

In [ ]:
best_logr = grid_logr.best_estimator_

In [ ]:
y_pred = best_logr.predict(X_test)

In [ ]:
y_pred_train = best_logr.predict(X_train)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
print(classification_report(y_train, y_pred_train))

In [ ]:
best_logr.coef_

In [ ]:
features_transformed